<a href="https://colab.research.google.com/github/yuki-shi/pokedex-flask/blob/main/serebii_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports !!!

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
from collections import OrderedDict

### Webscraping !!!

In [2]:
url = 'https://serebii.net/pokemon/gen1pokemon.shtml'
soup = BeautifulSoup(requests.get(url).text, 'html.parser')

Para a extração da tabela, começamos selecionando apenas os headers.

A estrutura HTML da página é um pouco bagunçada, mas, por sorte, todos os elementos do header estão com a classe *fooevo*.

Após isso, vamos criar uma lista com o innerText das tags scrapeadas, além de limpar cada elemento de seus caractéres de escape.

In [3]:
header = []
h = soup.find_all('td', class_='fooevo')

for i in h:
  header.append(i.text)

header = [x.strip('\r\n\t') for x in header] # visto que alguns possuíam caractéres de escape
header.remove('Base Stats') # a tabela é pivotada, vamos retirar o header sobrejascente
print(header)

['No.', 'Pic', 'Name', 'Type', 'Abilities', 'HP', 'Att', 'Def', 'S.Att', 'S.Def', 'Spd']


A partir do header que conseguimos, criamos um *dicionário ordenado* usando-os como keys.

Aqui é importante usarmos tal estrutura para tornar os próximos passos mais simples, mantendo-se a estrutura da tabela original.


Para cada key, criamos uma lista como valor.

In [4]:
poke_dict = OrderedDict.fromkeys(header)

for i in poke_dict.keys(): # talvez seja inútil, visto que usaremos .setdefault em seguida
  poke_dict[i] = []

print(poke_dict)

OrderedDict([('No.', []), ('Pic', []), ('Name', []), ('Type', []), ('Abilities', []), ('HP', []), ('Att', []), ('Def', []), ('S.Att', []), ('S.Def', []), ('Spd', [])])


Selecionamos todos os tags *tr* e alocamo-os na variável *tbody*, que sofrerá um slice a fim de retirar o header que já extraímos.

In [5]:
tbody = soup.find_all('tr')
tbody = tbody[2:]

In [6]:
for index, tr in enumerate(tbody):
  if index % 2 == 0:  # pela extração clunky, pegamos apenas os indexes pares >_>
    for i, key in enumerate(poke_dict.keys()):
      poke_dict.setdefault(key, []).append(tr.find_all('td', class_='fooinfo')[i].text.strip('\r\n\t'))

  Eis o famigerado datarame inicial:

In [7]:
df = pd.DataFrame(poke_dict)
df.head()

,No.,Pic,Name,Type,Abilities,HP,Att,Def,S.Att,S.Def,Spd
0,#001,,Bulbasaur,,Overgrow Chlorophyll,45,49,49,65,65,45
1,#002,,Ivysaur,,Overgrow Chlorophyll,60,62,63,80,80,60
2,#003,,Venusaur,,Overgrow Chlorophyll,80,82,83,100,100,80
3,#004,,Charmander,,Blaze Solar Power,39,52,43,60,50,65
4,#005,,Charmeleon,,Blaze Solar Power,58,64,58,80,65,80


### API !!!